In [1]:
#imports and global variables

%run SharedParams.ipynb
%run Fri.ipynb
%run MerkleTree.ipynb
fib_a = F_gen

fiat_shamir = FiatShamir()
proof_data = []
commitments = []

In [2]:
# generate trace polynomial (f)

fib_points = []
c = F_gen
d = F_gen
for i in range(last_fib_index + 1):
    fib_points.append((g**(RS_factor*i), c))
    c, d = d, c+d

#fib_points[800] = (g**(RS_factor*800), 3) #mistaken trace for test
trace_poly = PR.lagrange_polynomial(fib_points)

In [3]:
# generate trace polynomial evauluation over g<w>

coset_eval = []
for i in range(g_order):
    coset_eval.append(trace_poly(w * g**i))


In [4]:
# generate fibonacci constraint polynomial

fibonacci_constraint_trace_poly = trace_poly(PR(x)) + trace_poly(gamma * PR(x)) - trace_poly(PR(x) * gamma ** 2)
fibonacci_constraint_missing_zeroes_poly = 1
for i in range(last_fib_index - 1, trace_size):
    fibonacci_constraint_missing_zeroes_poly *= (PR(x) - gamma**i)
# For optimization - change PR(x)**trace_size to a faster implementation
fibonacci_constraint_zeroes_poly = (PR(x)**trace_size - PR(1)) / fibonacci_constraint_missing_zeroes_poly
ld_fibonacci_constraint_trace_poly = fibonacci_constraint_trace_poly / fibonacci_constraint_zeroes_poly

In [5]:
# generate boundary constraint polynomials

boundary_constraint_1 = (trace_poly - fib_1)/(PR(x)-1)
boundary_constraint_last = (trace_poly - fib_last)/(PR(x) - gamma**last_fib_index)

In [6]:
# commit on trace polynomial evaluation

eval_merkle = MerkleTree(coset_eval)
proof_data.append(f"eval_merkle:{eval_merkle.root.hex()}")
commitments.append(eval_merkle.root)

In [7]:
# generate composition polynomial

beta0, beta1, beta2, beta3, beta4, beta5 = (fiat_shamir.get_random_int(commitments) % P for i in range(6))

cp = beta0 + beta1 * (ld_fibonacci_constraint_trace_poly * PR(x) ** (1023 - fib_constraint_max_deg))
cp += beta2 + beta3 * (boundary_constraint_1 * PR(x) ** (1023 - boundary_constraint_max_deg))
cp += beta4 + beta5 * (boundary_constraint_last * PR(x) ** (1023 - boundary_constraint_max_deg))

In [8]:
# generate cp evalutaions over w<g> coset
cp_eval = []
for i in range(g_order):
    cp_eval.append(cp(w * g**i))

cp_eval_merkle = MerkleTree(cp_eval)
proof_data.append(f"cp_eval_merkle:{cp_eval_merkle.root.hex()}")
commitments.append(cp_eval_merkle.root)

In [9]:
# generate OOODS parameters

oods_z = F_gen * fiat_shamir.get_random_int(commitments)

cp_z = cp(oods_z)
mask_z1 = trace_poly(oods_z)
mask_z2 = trace_poly(oods_z * gamma)
mask_z3 = trace_poly(oods_z * gamma**2)

proof_data.append(f"cp_z:{cp_z}")
commitments.append(cp_z)
proof_data.append(f"mask_z1:{mask_z1}")
commitments.append(mask_z1)
proof_data.append(f"mask_z2:{mask_z2}")
commitments.append(mask_z2)
proof_data.append(f"mask_z3:{mask_z3}")
commitments.append(mask_z3)

alpha1, alpha2, alpha3, alpha_cp = (fiat_shamir.get_random_int(commitments) for i in range(4))

In [10]:
oods_poly = alpha1 * (trace_poly-mask_z1)/(PR('x')-oods_z)
oods_poly += alpha2 * (trace_poly-mask_z2)/(PR('x')- (gamma * oods_z))
oods_poly += alpha3 * (trace_poly-mask_z3)/(PR('x')-(gamma ** 2 * oods_z))
oods_poly += alpha_cp * (cp-cp_z)/(PR('x')-oods_z)

In [11]:
# generate oods_poly evalutaions over w<g> coset
oods_poly_eval = []
for i in range(g_order):
    oods_poly_eval.append(oods_poly(w * g**i))

In [12]:
# generate fri proof and queries

fri = Fri(oods_poly_eval, fiat_shamir, proof_data)
for i in range(N_FRI_QUERIES):
    trace_index = fiat_shamir.get_random_int(commitments) % g_order
    
    # commit on trace_poly eval
    trace_data, trace_path = eval_merkle.get_path(trace_index % g_order)
    MerkleTree.auth_path(trace_data, trace_index, trace_path, eval_merkle.root, eval_merkle.tree_height)
    proof_data.append(f"trace_eval:{trace_data}")
    proof_data.append(f"path:{trace_path}")
    
    # commit on cp eval
    cp_data, cp_path = cp_eval_merkle.get_path(trace_index % g_order)
    MerkleTree.auth_path(cp_data, trace_index, cp_path, cp_eval_merkle.root, cp_eval_merkle.tree_height)
    proof_data.append(f"cp_eval:{cp_data}")
    proof_data.append(f"path:{cp_path}")

    fri.query_layers(trace_index, fiat_shamir, proof_data)

In [17]:
print(commitments)
print(proof_data)
